In [ ]:
import os
import sys
from logging import getLogger

import optuna
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, log_loss, precision_score, recall_score
# from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC, LinearSVC
from torch import cuda
from xgboost import XGBClassifier

sys.path.append(os.getcwd() + "/../")

# from config.params import ML_MODEL_DIR, ML_RESULT_DIR, dict_to_str
from MachineLearning.basemodel import BaseModel


In [ ]:
class TuningOptuna:
    def __init__(self) -> None:
        self.best_parameter_dict = {}

    def load(self, mode="mix", parameter="enstrophy", label=1, randomstate=42):
        self.mode = mode
        self.parameter = parameter

        if mode == "sep":
            self.label = label
        else:
            self.label = None

        self.model = BaseModel.load_npys(mode=mode, parameter=parameter, label=label, random_state=randomstate)
        self.eval_set = [(self.model.X_train, self.model.y_train)]

        return self.model

    def set_params(self, trial):
        if cuda.is_available():
            tree_methods = "gpu_hist"
        else:
            tree_methods = "hist"

        tuning_params = {
            "max_depth": trial.suggest_int("max_depth", 1, 10),
            "eta": trial.suggest_float("eta", 0.001, 0.3),
            "gamma": trial.suggest_float("gamma", 0.01, 10),
            "min_child_weight": trial.suggest_float("min_child_weight", 0.01, 10),
            "subsample": trial.suggest_float("subsample", 0.2, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 2.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 2.0),
            "tree_method": tree_methods,
            "n_estimators": 500,
            "n_jobs": -1,
            "random_state": 42,
            "eval_metric": "auc",
        }

        return tuning_params

    def _objective(self, trial):
        tuning_params = self.set_params(trial)
        clf = XGBClassifier(**tuning_params)
        params = {
            "verbosity": 0,
            "early_stopping_rounds": 10
        }
        clf.set_params(**params)

        clf.fit(self.model.X_train, self.model.y_train, eval_set=self.eval_set)

        # 目的関数用にAUCを算出
        pred_test = clf.predict(self.model.X_test)

        # 目的関数は(1-AUC)の最小化と定義
        return accuracy_score(self.model.y_test, pred_test)

    def learning(self):
        self.study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
        self.study.optimize(lambda x: self._objective(x), n_trials=50)

    def save(self):
        # 最適パラメータの表示と保持
        best_params = self.study.best_trial.params
        # best_score = self.study.best_trial.value
        self.best_parameter_dict[self.parameter] = best_params


In [ ]:
# ベイズ最適化を実行
mode = "mix"  # sep, mixsep, mix
# parameter = "density"  # density, energy, enstrophy, pressure, magfieldx, magfieldy, velocityx, velocityy
label = 2

tu = TuningOptuna()
for parameter in ["density", "energy", "enstrophy", "pressure", "magfieldx", "magfieldy", "velocityx", "velocityy"]:
# for parameter in ["energy"]:
    tu.load(mode=mode, parameter=parameter, label=label)
    tu.learning()

    tu.save()


In [ ]:
tu.best_parameter_dict